zaczytanie niezbędnych bibliotek

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.preprocessing import StandardScaler

zaczytaj dane dotyczace sprzedazy

In [2]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "Salary Data.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "rkiattisak/salaly-prediction-for-beginer",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

<ipython-input-2-72cc2aa30810>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


100%|██████████| 18.9k/18.9k [00:00<00:00, 26.0MB/s]

First 5 records:     Age  Gender Education Level          Job Title  Years of Experience  \
0  32.0    Male      Bachelor's  Software Engineer                  5.0   
1  28.0  Female        Master's       Data Analyst                  3.0   
2  45.0    Male             PhD     Senior Manager                 15.0   
3  36.0  Female      Bachelor's    Sales Associate                  7.0   
4  52.0    Male        Master's           Director                 20.0   

     Salary  
0   90000.0  
1   65000.0  
2  150000.0  
3   60000.0  
4  200000.0  


preprocessing

1.   sprawdż typy danych
2.   zmien etykiedy na zmienne liczbowe
3. w przypadku brakujacych danych uzupelnij (w przypadku etykiet) najczesciej wystepującą wartością lub średnią wartością



In [4]:
print(df.info())  # Wyświetla informacje o typach danych i brakujących wartościach

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  373 non-null    float64
 1   Gender               373 non-null    object 
 2   Education Level      373 non-null    object 
 3   Job Title            373 non-null    object 
 4   Years of Experience  373 non-null    float64
 5   Salary               373 non-null    float64
dtypes: float64(3), object(3)
memory usage: 17.7+ KB
None


In [5]:
from sklearn.preprocessing import LabelEncoder

# Inicjalizacja encodera
label_encoder = LabelEncoder()

# Przekształcenie kolumn kategorycznych na liczby
for column in df.columns:
  if df[column].dtype == 'object':
    df[column] = label_encoder.fit_transform(df[column])

print(df.head())  # Podgląd danych po transformacji

    Age  Gender  Education Level  Job Title  Years of Experience    Salary
0  32.0       1                0        159                  5.0   90000.0
1  28.0       0                1         17                  3.0   65000.0
2  45.0       1                2        130                 15.0  150000.0
3  36.0       0                0        101                  7.0   60000.0
4  52.0       1                1         22                 20.0  200000.0


Podziel zbiór danych na testowy i treningowy w stosunku 30/70

In [6]:
from sklearn.model_selection import train_test_split

# Definiujemy cechy (X) i zmienną docelową (y)
X = df.drop(columns=["Salary"])  # Wszystkie kolumny poza Salary to cechy
y = df["Salary"]  # Zmienna docelowa

# Podział na zbiór treningowy (70%) i testowy (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Sprawdzenie rozmiaru zbiorów
print(f"Rozmiar zbioru treningowego: {X_train.shape[0]} próbek")
print(f"Rozmiar zbioru testowego: {X_test.shape[0]} próbek")

Rozmiar zbioru treningowego: 262 próbek
Rozmiar zbioru testowego: 113 próbek


Stworz model regresji liniowej korzystajac z parametrów domyślnych
Przykład dokumentacji: https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py

In [11]:
from sklearn.linear_model import LinearRegression
# Import SimpleImputer from sklearn.impute
from sklearn.impute import SimpleImputer

# Create an imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')  # You can use 'median' or 'most_frequent' as well

# Fit the imputer on the training data and transform both training and testing data
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

imputer_y = SimpleImputer(strategy='mean')

y_train = imputer_y.fit_transform(y_train.values.reshape(-1, 1)) # Reshape to 2D for imputer
y_train = y_train.ravel()

# Now you can train your model
model = LinearRegression()
model.fit(X_train, y_train)

# Predykcja na zbiorze testowym
y_pred = model.predict(X_test)

# Wyświetlenie współczynników regresji
print("Współczynniki regresji:", model.coef_)
print("Wartość intercept (punkt przecięcia):", model.intercept_)

Współczynniki regresji: [ 3086.3742258   5942.18491947 13933.42430929    25.56102669
  2686.85296315]
Wartość intercept (punkt przecięcia): -55422.089219414614


In [13]:
import pandas as pd
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Create an imputer to fill missing values with the mean
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on the data and transform it
X_imputed = imputer.fit_transform(X)

# calculating VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_imputed, i)
                  for i in range(len(X.columns))]
print(vif_data)

               feature        VIF
0                  Age  15.129201
1               Gender   2.047736
2      Education Level   2.330338
3            Job Title   4.892569
4  Years of Experience   9.426853


ocen model na zbiorze testowym za pomocą miar: średni błąd kwadratowy, średni błąd absolutny, wynik wyjaśnionej wariancji

In [18]:
y_test = imputer_y.transform(y_test.reshape(-1, 1))
y_test = y_test.ravel()

# Obliczanie metryk oceny modelu
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)


print("Średni błąd kwadratowy (MSE):", mse)
print("Średni błąd absolutny (MAE):", mae)
print("Wynik R^2:", r2)
print("Wynik wyjaśnionej wariancji (EVS):", evs)

Średni błąd kwadratowy (MSE): 251867808.61881605
Średni błąd absolutny (MAE): 10980.545681608131
Wynik R^2: 0.8819044534150401
Wynik wyjaśnionej wariancji (EVS): 0.8827891675163093


spróbuj poprawić model, zacznij od danych, sprawdz korelacje, znormalizuj

Zbuduj model regresji odpornej i przetestuj jego predykcje

Wykreś przewidziane i faktyczne wartości obu modeli (przykład w dokumentacji: https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_regressor.html#sphx-glr-auto-examples-ensemble-plot-voting-regressor-py)

In [20]:
# 1. Check for Correlations
correlation_matrix = df.corr()
print("Correlation Matrix:\n", correlation_matrix)

# Identify highly correlated features (e.g., correlation > 0.8 or < -0.8)
# You might need to manually inspect the correlation matrix or set a threshold

# 2. Normalize the Data
# Separate features (X) and target (y)
X = df.drop(columns=["Salary"])
y = df["Salary"]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Impute missing values before scaling
imputer = SimpleImputer(strategy='mean')  # or 'median', 'most_frequent'
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Apply StandardScaler for normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. Re-train the Model
model = LinearRegression()
model.fit(X_train, y_train)

# 4. Evaluate the Improved Model
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("\nModel Evaluation with Normalization:")
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)
print("EVS:", evs)

Correlation Matrix:
                           Age    Gender  Education Level  Job Title  \
Age                  1.000000 -0.020324         0.562693   0.117629   
Gender              -0.020324  1.000000         0.008677   0.041562   
Education Level      0.562693  0.008677         1.000000   0.127791   
Job Title            0.117629  0.041562         0.127791   1.000000   
Years of Experience  0.979128  0.002884         0.590863   0.100162   
Salary               0.922335  0.071106         0.670371   0.136206   

                     Years of Experience    Salary  
Age                             0.979128  0.922335  
Gender                          0.002884  0.071106  
Education Level                 0.590863  0.670371  
Job Title                       0.100162  0.136206  
Years of Experience             1.000000  0.930338  
Salary                          0.930338  1.000000  


ValueError: Input y contains NaN.

Zmodyfikuj kod, zeby zwizualizować wynik lepszego z modeli

In [ ]:
import matplotlib.pyplot as plt

from sklearn.metrics import PredictionErrorDisplay

fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
PredictionErrorDisplay.from_predictions(
    y,
    y_pred=y_pred,
    kind="actual_vs_predicted",
    subsample=100,
    ax=axs[0],
    random_state=0,
)
axs[0].set_title("Actual vs. Predicted values")
PredictionErrorDisplay.from_predictions(
    y,
    y_pred=y_pred,
    kind="residual_vs_predicted",
    subsample=100,
    ax=axs[1],
    random_state=0,
)
axs[1].set_title("Residuals vs. Predicted Values")
fig.suptitle("Plotting cross-validated predictions")
plt.tight_layout()
plt.show()